In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

In [ ]:
df = pd.read_csv("../data/customer_intelligence_dataset.csv")
print("Rows, Cols:", df.shape)
display(df.head(10))
print("\nColumns:", list(df.columns))
print("\nDuplicate rows:")
print(df.duplicated().sum())
print("\nDescription:", df.info())


In [ ]:
print("\nData types:")
print(df.dtypes)
print("\nMissing values per column:")
print(df.isna().sum().sort_values(ascending=False))
df = df.drop(columns=["total_value"])
df['total_value'] = df['price'] * df['quantity']
df["total_value_log"] = np.log1p(df["total_value"])

In [ ]:
for c in ["sale_date", "last_purchase_date"]:
    df[c] = pd.to_datetime(df[c], errors="coerce")

df["sale_year"]    = df["sale_date"].dt.year.astype("Int16")
df["sale_month"]   = df["sale_date"].dt.month.astype("Int8")
df["sale_day"]     = df["sale_date"].dt.day.astype("Int8")
df["sale_dow"]     = df["sale_date"].dt.dayofweek.astype("Int8")  # 0=Mon ... 6=Sun
df["is_weekend"]   = (df["sale_dow"] >= 5).astype("uint8")    

for c in ["sale_id", "customer_id", "product_id", "product_name"]:
    df[c] = df[c].astype("string")

for c in ["gender", "region", "segment", "category", "sentiment"]:
    df[c] = df[c].astype("category")



In [ ]:
assert set(df["churn"].unique()) <= {0, 1}, f"Unexpected churn values: {df['churn'].unique()}"
df["churn"] = df["churn"].astype("uint8")
df.head()

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print("\nNumeric columns:", num_cols)
display(df[num_cols].describe().T if num_cols else "No numeric columns found.")


In [ ]:
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
print("\nCategorical columns:", cat_cols)
for c in cat_cols[:5]:
    print(f"\nTop 10 values for '{c}':")
    print(df[c].value_counts(dropna=False).head(10))

In [ ]:

top_customers = df.groupby('customer_id')['total_value'].sum().sort_values(ascending=False).head(10)
print("Top 10 highest paying customers:")
display(top_customers)

In [ ]:
for c in num_cols:
    plt.figure(figsize=(6,4))
    sns.histplot(df[c].dropna(), bins=30, kde=False)
    plt.title(f"Distribution: {c}")
    plt.xlabel(c); plt.ylabel("Count")
    plt.tight_layout()
    plt.show()

In [ ]:
if num_cols:
    for col in num_cols:
        plt.figure(figsize=(6, 4))
        sns.boxplot(data=df, x=col, orient="h")
        plt.title(f"Boxplot: {col}")
        plt.tight_layout()
        plt.show()

In [ ]:

df.head()

In [ ]:
if len(num_cols) >= 2:
    corr = df[num_cols].corr(numeric_only=True)
    plt.figure(figsize=(6,5))
    sns.heatmap(corr, annot=True, cmap='coolwarm')
    plt.title("Correlation Heatmap (numeric)")
    plt.tight_layout()
    plt.show()

In [ ]:
df.head(10)

In [ ]:
import pandas as pd
import numpy as np
import nltk

df_fe = df.copy()

assert pd.api.types.is_datetime64_any_dtype(df_fe["sale_date"]), "sale_date must be datetime"
assert "total_value" in df_fe.columns, "total_value missing"
assert "total_value_log" in df_fe.columns, "total_value_log missing"
assert "is_weekend" in df_fe.columns and "sale_dow" in df_fe.columns, "derive date features first"

AS_OF_DATE = df_fe["sale_date"].max()
HIGH_TICKET_THR = df_fe["total_value"].quantile(0.90)

df_fe["feedback_text"] = df_fe.get("feedback_text", "").fillna("").astype(str).str.strip()
df_fe["has_feedback"] = (df_fe["feedback_text"].str.len() > 0).astype("uint8")

try:
    from nltk.sentiment import SentimentIntensityAnalyzer
except LookupError:
    nltk.download("vader_lexicon")
    from nltk.sentiment import SentimentIntensityAnalyzer
try:
    _ = nltk.data.find("sentiment/vader_lexicon.zip")
except LookupError:
    nltk.download("vader_lexicon")

sia = SentimentIntensityAnalyzer()

def _score_text(t: str) -> float:
    return float(sia.polarity_scores(t)["compound"]) if t else 0.0

df_fe["sentiment_score"] = df_fe["feedback_text"].map(_score_text)

def _label_from_score(s: float) -> str:
    if s >= 0.30:   return "Positive"
    if s <= -0.30:  return "Negative"
    return "Neutral"

df_fe["sentiment_pred"] = df_fe["sentiment_score"].map(_label_from_score).astype("category")

sent_map = {"positive":"Positive", "negative":"Negative", "neutral":"Neutral"}
df_fe["sentiment_norm"] = np.where(
    df_fe["has_feedback"].eq(1),
    df_fe["sentiment_pred"].astype(str),
    np.nan,
)
if "sentiment" in df_fe.columns:
    df_fe.loc[df_fe["sentiment_norm"].isna(), "sentiment_norm"] = (
        df_fe["sentiment"].astype(str).str.lower().map(sent_map)
    )
df_fe["sentiment_norm"] = df_fe["sentiment_norm"].fillna("NoFeedback").astype("category")

df_fe["is_high_ticket"] = (df_fe["total_value"] > HIGH_TICKET_THR).astype("uint8")

def build_customer_features(g: pd.DataFrame) -> pd.Series:
    first_sale = g["sale_date"].min()
    last_sale  = g["sale_date"].max()
    orders     = int(len(g))

    m_sum   = float(g["total_value"].sum())
    m_med   = float(g["total_value"].median())
    m_max   = float(g["total_value"].max())
    q_mean  = float(g["quantity"].mean())
    q_max   = float(g["quantity"].max())
    p_mean  = float(g["price"].mean())

    dow_dist = g["sale_dow"].value_counts(normalize=True).reindex(range(7), fill_value=0.0)

    uniq_cat = int(g["category"].nunique())
    uniq_prod = int(g["product_id"].nunique())
    vc_cat = g["category"].value_counts(normalize=True)
    top_cat = vc_cat.index[0] if len(vc_cat) else np.nan
    top_cat_share = float(vc_cat.iloc[0]) if len(vc_cat) else 0.0

    with_fb = g[g["has_feedback"] == 1]
    pos = int((with_fb["sentiment_norm"] == "Positive").sum())
    neg = int((with_fb["sentiment_norm"] == "Negative").sum())
    neu = int((with_fb["sentiment_norm"] == "Neutral").sum())
    den = max(1, pos + neg + neu)
    neg_rate = float(neg / den)

    last_sent = g.sort_values("sale_date")["sentiment_norm"].iloc[-1] if len(g) else "NoFeedback"

    recent_mask = g["sale_date"] >= (AS_OF_DATE - pd.Timedelta(days=90))
    recent_neg_flag = int(((g.loc[recent_mask & (g["has_feedback"] == 1), "sentiment_norm"] == "Negative").sum()) > 0)

    feedback_count = int((g["has_feedback"] == 1).sum())
    feedback_rate = float(feedback_count / max(1, len(g)))

    g_sorted = g.sort_values("sale_date")
    age_latest = int(g_sorted["age"].iloc[-1])
    gender_mode = g["gender"].mode(dropna=False)
    gender_final = gender_mode.iloc[0] if not gender_mode.empty else np.nan
    region_mode = g["region"].mode(dropna=False)
    region_final = region_mode.iloc[0] if not region_mode.empty else np.nan

    tenure_days = int((last_sale - first_sale).days)
    recency_days = int((AS_OF_DATE - last_sale).days)

    high_ticket_rate = float(g["is_high_ticket"].mean())

    return pd.Series({
        "orders": orders,
        "first_sale": first_sale,
        "last_sale": last_sale,
        "tenure_days": tenure_days,
        "recency_days": recency_days,

        "monetary_sum": m_sum,
        "monetary_median": m_med,
        "monetary_max": m_max,

        "avg_quantity": q_mean,
        "max_quantity": q_max,
        "avg_price": p_mean,

        "dow_0_rate": float(dow_dist.loc[0]),
        "dow_1_rate": float(dow_dist.loc[1]),
        "dow_2_rate": float(dow_dist.loc[2]),
        "dow_3_rate": float(dow_dist.loc[3]),
        "dow_4_rate": float(dow_dist.loc[4]),
        "dow_5_rate": float(dow_dist.loc[5]),
        "dow_6_rate": float(dow_dist.loc[6]),

        "unique_categories": uniq_cat,
        "unique_products": uniq_prod,
        "top_category": top_cat,
        "top_category_share": top_cat_share,

        "sent_pos": pos,
        "sent_neg": neg,
        "sent_neu": neu,
        "neg_rate": neg_rate,
        "last_sentiment": last_sent,
        "recent_neg_flag": recent_neg_flag,
        "feedback_count": feedback_count,
        "feedback_rate": feedback_rate,

        "age_latest": age_latest,
        "gender": gender_final,
        "region": region_final,

        "high_ticket_rate": high_ticket_rate,
    })

gb = df_fe.groupby("customer_id", group_keys=False)
try:
    cust = gb.apply(build_customer_features, include_groups=False).reset_index()
except TypeError:
    cust = gb.apply(build_customer_features).reset_index()

y_cust = df_fe.groupby("customer_id")["churn"].max().rename("churn")
cust = cust.merge(y_cust, on="customer_id", how="left")

days = cust["tenure_days"].clip(lower=1)
cust["aov"] = cust["monetary_sum"] / cust["orders"].clip(lower=1)
cust["orders_per_30d"]   = cust["orders"] / (days / 30.0)
cust["monetary_per_30d"] = cust["monetary_sum"] / (days / 30.0)

cust.replace([np.inf, -np.inf], np.nan, inplace=True)
cust[["orders_per_30d","monetary_per_30d"]] = cust[["orders_per_30d","monetary_per_30d"]].fillna(0)

for c in ["gender", "region", "top_category", "last_sentiment"]:
    cust[c] = cust[c].astype("category")

cust_ml = pd.get_dummies(
    cust,
    columns=["gender", "region", "top_category", "last_sentiment"],
    drop_first=True,
    dtype=np.uint8
    )

drop_cols = [
    "first_sale", "last_sale",
    "dow_0_rate",
    "monetary_mean"
    ]
cust_ml.drop(columns=[c for c in drop_cols if c in cust_ml.columns], inplace=True, errors="ignore")

print(cust_ml.head(10))
print(f"Null vals: {cust_ml.isnull().sum()}")
print(f"Duplicate vals: {cust_ml.duplicated().sum()}")

X = cust_ml.drop(columns=["churn"], errors="ignore")
y = cust_ml["churn"].astype("uint8")
print("Churn Rate: ", y.mean())

print("Customers:", cust_ml.shape[0], "| Features:", X.shape[1])
print("Sample columns:", list(X.columns)[:20], "...")
X.corr(numeric_only=True).abs().unstack().sort_values(ascending=False).drop_duplicates().head(15)

In [ ]:
linear_drop = [
    "aov", "monetary_median", "monetary_max", "avg_price",
    "unique_products", "max_quantity", "unique_categories"
 ]
X_linear = X.drop(columns=[c for c in linear_drop if c in X.columns])

# Outlier capping is commented out below
# for c in ["orders_per_30d", "monetary_per_30d"]:
#     p99 = X[c].quantile(0.99)
#     X[c] = X[c].clip(upper=p99)
#     if "X_linear" in globals() and c in X_linear.columns:
#         X_linear[c] = X_linear[c].clip(upper=p99)


In [ ]:
# === Save engineered dataframes for modeling ===
cust_ml.to_csv("../data/customer_snapshot_ml.csv", index=False)
X_linear.to_csv("../data/customer_snapshot_Xlinear.csv", index=False)
print("Saved: customer_snapshot_ml.csv and customer_snapshot_Xlinear.csv")